In [1]:
import os
os.chdir('../../..')
import sys
# sys.path.append("/workspace/src")

In [2]:
import torch

from databases.datasets import (
    Mpi3dTestDataset,
    Mpi3dTrainDataset,
    PersonStackedMucoTempDataset,
    ConcatPoseDataset,
)
from model.videopose import TemporalModel, TemporalModelOptimized1f
from training.preprocess import *
from training.loaders import ChunkedGenerator, UnchunkedGenerator
from training.callbacks import TemporalMupotsEvaluator, TemporalTestEvaluator
from scripts.eval import load_model
from util.geom import orient2pose

In [3]:
exp_key = "21a1850a239642e4978f2cfa0756803f"
layernorm = "batchnorm"
ordered_batch = False
output_path = f"../models/{exp_key}"

_config = {
    "num_epochs": 15,
    "preprocess_2d": "DepthposeNormalize2D",
    "preprocess_3d": "SplitToRelativeAbsAndMeanNormalize3D",
    "shuffle": True,
    "ordered_batch": ordered_batch,
    # training
    "optimiser": "adam",
    "adam_amsgrad": True,
    "learning_rate": 1e-5,
    "sgd_momentum": 0,
    "batch_size": 1024,
    "train_time_flip": True,
    "test_time_flip": True,
    "lr_scheduler": {
        "type": "multiplicative",
        "multiplier": 0.95,
        "step_size": 1,
    },
    # dataset
    "ignore_invisible": True,
    "train_data": "mpii_train",  # +muco
    "pose2d_type": "hrnet",
    "pose3d_scaling": "normal",
    "megadepth_type": "megadepth_at_hrnet",
    "cap_25fps": True,
    "stride": 2,
    "simple_aug": True,  # augments data by duplicating each frame
    "model": {
        "loss": "orient",
        "orient_loss": "proj",
        "channels": 512,
        "dropout": 0.25,
        "filter_widths": [3, 3, 3],
        "layernorm": layernorm,  # False,
    },
    "orient_norm": None,
}

In [4]:
# train_data = Mpi3dTrainDataset(
#     _config["pose2d_type"],
#     _config["pose3d_scaling"],
#     _config["cap_25fps"],
#     _config["stride"],
# )
# test_data = Mpi3dTestDataset(
#     _config["pose2d_type"], _config["pose3d_scaling"], eval_frames_only=True
# )

In [5]:
c, model = load_model(output_path)

In [6]:
dataset = Mpi3dTestDataset(
    _config["pose2d_type"],
    _config.get("pose3d_scaling", "normal"),
    eval_frames_only=True,
)
params_path = os.path.join(output_path, "preprocess_params.pkl")
transform = SaveableCompose.from_file(params_path, dataset, globals())
dataset.transform = transform

assert isinstance(transform.transforms[1].normalizer, MeanNormalize3D)
normalizer3d = transform.transforms[1].normalizer

post_process_func = get_postprocessor(_config, dataset, normalizer3d)

In [7]:
augment = True
pad = (model.receptive_field() - 1) // 2
generator = UnchunkedGenerator(dataset, pad, augment)
seqs = sorted(np.unique(dataset.index.seq))

data_3d_mm = {}
preprocessed3d = {}
for seq in seqs:
    inds = np.where(dataset.index.seq == seq)[0]
    batch = dataset.get_samples(inds, False)
    preprocessed3d[seq] = batch["pose3d"][batch["valid_pose"]]
    data_3d_mm[seq] = dataset.poses3d[inds][batch["valid_pose"]]
#     break

bl = {}
root = {}
org_pose3d = {}
for seq in seqs:
    inds = np.where(dataset.index.seq == seq)[0]
    batch = dataset.get_samples(inds, False)
    bl[seq] = batch["length"][batch["valid_pose"]]
    root[seq] = batch["root"][batch["valid_pose"]]
    org_pose3d[seq] = batch["org_pose3d"][batch["valid_pose"]]
#     break

In [11]:
preds = {}
raw_preds = {}
losses = {}
with torch.no_grad():
    for i, (pose2d, valid) in enumerate(generator):
        seq = seqs[i]
        pred3d = (
            model(torch.from_numpy(pose2d).cuda()).detach().cpu().numpy()
        )
        raw_preds[seq] = pred3d.copy()  # .cpu().numpy()

        valid = valid[0]
        pred_bo_np = pred3d[0][valid].reshape([-1, 2, 16])
#         if orient_norm is None:
#             pass
#         elif orient_norm == "_1_1":
#             pred_bo_np *= np.pi
#         elif orient_norm == "0_1":
#             pred_bo_np = (pred_bo_np * 2 * np.pi) - np.pi
#         else:
#             raise Exception(
#                 f"Not supported oreitation norm: {self.orient_norm}"
#             )
        pred_bo = torch.from_numpy(pred_bo_np).to("cuda")
        orient_pred3d = (
            orient2pose(
                pred_bo,
                # torch.from_numpy(self.bo[seq]).to("cuda"),
                torch.from_numpy(bl[seq]).to("cuda"),
                torch.from_numpy(root[seq]).to("cuda"),
            )
            .cpu()
            .numpy()
        )
        preds[seq] = orient_pred3d
#         break

In [13]:
for seq in seqs:
    print(np.mean(preds[seq][:, 14, :] - org_pose3d[seq][:, 14, :]))

2.6229751101363753e-07
-7.202047561358453e-07
2.1947170727953678e-06
2.5228155686955436e-08
-7.918924450979476e-07
4.554259953007117e-06


In [9]:
p = load('../models/21a1850a239642e4978f2cfa0756803f/test_results.pkl')

In [10]:
gt = p['pose3d']
pred = p['pred']

In [19]:
np.sum([preds[seq].shape[0] for seq in seqs])

2860

In [16]:
pred.shape

(24888, 17, 3)

In [15]:
np.mean(pred[:, 14, :] - gt[:, 14, :])

-876.1850995647015